In [ ]:
---

SELECT COUNT(status)
FROM company
WHERE status = 'closed';

---

SELECT funding_total
FROM company
WHERE category_code = 'news'
AND country_code = 'USA'
ORDER BY funding_total DESC;

---

SELECT SUM(price_amount)
FROM acquisition
WHERE term_code = 'cash'
AND EXTRACT(YEAR FROM acquired_at) BETWEEN 2011 AND 2013;

---

SELECT first_name,
       last_name,
       twitter_username
FROM people
WHERE twitter_username LIKE 'Silver%';

---

SELECT *
FROM people
WHERE twitter_username LIKE '%money%'
AND last_name LIKE 'K%';

---

SELECT country_code,
    SUM(funding_total)
FROM company
GROUP BY country_code
ORDER BY SUM(funding_total) DESC;

---

SELECT funded_at,
    MIN(raised_amount),
    MAX(raised_amount)
FROM funding_round
GROUP BY funded_at
HAVING MIN(raised_amount) > 0 AND MIN(raised_amount) != MAX(raised_amount);

---

SELECT *,
   CASE 
       WHEN invested_companies >= 100 THEN 'high_activity'
       WHEN invested_companies >= 20 AND invested_companies < 100 THEN 'middle_activity'
       WHEN invested_companies < 20 THEN 'low_activity'
   END
FROM fund;

---


SELECT 
       CASE
           WHEN invested_companies>=100 THEN 'high_activity'
           WHEN invested_companies>=20 THEN 'middle_activity'
           ELSE 'low_activity'
       END AS activity,
              ROUND(AVG(investment_rounds))
FROM fund
GROUP BY activity
ORDER BY AVG(investment_rounds);

---

SELECT country_code,
       MIN(invested_companies),
       MAX(invested_companies),
       AVG(invested_companies)
       
FROM fund
WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) BETWEEN 2010 AND 2012
GROUP BY country_code
HAVING MIN(invested_companies) > 0
ORDER BY AVG(invested_companies) DESC, country_code
LIMIT 10;

---

SELECT p.first_name,
       p.last_name,
       e.instituition
FROM people AS p
LEFT OUTER JOIN education AS e ON p.id = e.person_id;

---

SELECT c.name,
  COUNT(DISTINCT e.instituition)
FROM company AS c
 JOIN people AS p ON c.id = p.company_id
 JOIN education AS e ON p.id = e.person_id
GROUP BY c.name
ORDER BY COUNT(e.instituition) DESC
LIMIT 5;

---

SELECT DISTINCT c.name
FROM company AS c
JOIN funding_round AS fa ON c.id = fa.company_id
WHERE c.status LIKE '%closed%'
AND fa.is_first_round = 1 
AND fa.is_last_round = 1;

---

SELECT p.id
FROM people AS p
JOIN company AS c ON p.company_id = c.id
WHERE c.name IN 
                      (SELECT DISTINCT c.name
                       FROM company AS c
                       JOIN funding_round AS fa ON c.id = fa.company_id
                       WHERE c.status LIKE '%closed%'
                       AND fa.is_first_round = 1 
                       AND fa.is_last_round = 1);
    
---

SELECT DISTINCT p.id,
                e.instituition
FROM company AS c
INNER JOIN people AS p ON c.id = p.company_id
LEFT JOIN education AS e ON p.id = e.person_id
WHERE c.status LIKE 'closed'
   AND c.id IN (SELECT company_id
                FROM funding_round
                WHERE is_first_round = 1
                   AND is_last_round = 1)
   AND  e.instituition IS NOT NULL;

---

SELECT DISTINCT p.id,
                COUNT(e.instituition)
FROM company AS c
INNER JOIN people AS p ON c.id = p.company_id
LEFT JOIN education AS e ON p.id = e.person_id
WHERE c.status LIKE 'closed'
   AND c.id IN (SELECT company_id
                FROM funding_round
                WHERE is_first_round = 1
                   AND is_last_round = 1)
   AND  e.instituition IS NOT NULL
GROUP BY p.id;

---

SELECT AVG(filt.cnt)
FROM

(SELECT p.id,
       COUNT(e.instituition) AS cnt
FROM company AS c
INNER JOIN people AS p ON c.id = p.company_id
LEFT JOIN education AS e ON p.id = e.person_id
WHERE c.status LIKE 'closed'
   AND c.id IN (SELECT company_id
                FROM funding_round
                WHERE is_first_round = 1
                   AND is_last_round = 1)
   AND  e.instituition IS NOT NULL
GROUP BY p.id) AS filt;

---

SELECT AVG(filt.cnt)
FROM

(SELECT p.id,
       COUNT(e.instituition) AS cnt
FROM company AS c
INNER JOIN people AS p ON c.id = p.company_id
LEFT JOIN education AS e ON p.id = e.person_id
WHERE c.id IN (SELECT company_id
                FROM funding_round
                WHERE c.name = 'Facebook')
                   
   AND  e.instituition IS NOT NULL
GROUP BY p.id) AS filt;

---

SELECT f.name AS name_of_fund,
       c.name AS name_of_company,
       fr.raised_amount AS amount

FROM investment AS i
JOIN company AS c ON i.company_id = c.id
JOIN fund AS f ON i.fund_id = f.id
JOIN funding_round AS fr ON i.funding_round_id = fr.id
WHERE (EXTRACT(YEAR FROM funded_at) BETWEEN 2012 AND 2013)
AND c.milestones > 6;

---

SELECT company.name AS acquiring_company,
       tab2.price_amount,
       tab2.acquired_company,
       tab2.funding_total,
       ROUND(tab2.price_amount / tab2.funding_total)
FROM
(
    SELECT c.name AS acquired_company,
           c.funding_total,
           tab1.acquiring_company_id,
           tab1.price_amount
    FROM company AS c
    RIGHT JOIN (
                SELECT acquiring_company_id,
                       acquired_company_id,
                       price_amount
                FROM acquisition
                WHERE price_amount > 0
               ) AS tab1 ON c.id = tab1.acquired_company_id
 ) AS tab2 LEFT JOIN company ON company.id  = tab2.acquiring_company_id
WHERE tab2.funding_total > 0
ORDER BY  tab2.price_amount DESC, tab2.acquired_company
LIMIT 10;

---

SELECT c.name,
       EXTRACT(MONTH FROM funded_at)
FROM company AS c
JOIN funding_round AS fr ON c.id = fr.company_id
WHERE c.category_code = 'social'
AND EXTRACT(YEAR FROM fr.funded_at) BETWEEN 2010 AND 2013
AND fr.raised_amount > 0;

---

WITH
tab1 AS (SELECT EXTRACT(MONTH FROM funded_at) AS month,
                id AS funding_round_id
                FROM funding_round
         WHERE EXTRACT(YEAR FROM funded_at) BETWEEN 2010 AND 2013
         ),

tab2 AS (SELECT EXTRACT(MONTH FROM acquired_at) AS month,
                COUNT(acquired_company_id) AS count_acquired,
                SUM(price_amount) AS total_amount
         FROM acquisition
         WHERE EXTRACT(YEAR FROM acquired_at) BETWEEN 2010 AND 2013
         GROUP BY EXTRACT(MONTH FROM acquired_at)
        ),

tab3 AS (SELECT i.funding_round_id,
                f.name
        FROM investment AS i
        JOIN fund AS f ON f.id = i.fund_id
        WHERE fund_id IN (SELECT id
                          FROM fund
                          WHERE country_code LIKE 'USA')
        ),

tab4 AS (SELECT month,
                COUNT(DISTINCT name) AS count_USA
         FROM tab1 
         LEFT JOIN tab3 ON tab1.funding_round_id = tab3.funding_round_id
         GROUP BY month)
         
SELECT tab4.month,
       tab4.count_USA,
       tab2.count_acquired,
       tab2.total_amount
FROM tab4 
LEFT JOIN tab2 ON tab4.month = tab2.month;

---

WITH tab1 AS  (SELECT c.country_code AS country,
               AVG(c.funding_total) AS avg_total
            FROM company AS c
            WHERE EXTRACT(YEAR FROM c.founded_at) = 2011
            GROUP BY c.country_code, EXTRACT(YEAR FROM c.founded_at)),
            
tab2 AS (SELECT c.country_code AS country,
               AVG(c.funding_total) AS avg_total
            FROM company AS c
            WHERE EXTRACT(YEAR FROM c.founded_at) = 2012
            GROUP BY c.country_code, EXTRACT(YEAR FROM c.founded_at)),
            
tab3 AS (SELECT c.country_code AS country,
               AVG(c.funding_total) AS avg_total
            FROM company AS c
            WHERE EXTRACT(YEAR FROM c.founded_at) = 2013
            GROUP BY c.country_code, EXTRACT(YEAR FROM c.founded_at))
            
SELECT tab3.country,
       tab1.avg_total AS tatal_11,
       tab2.avg_total AS total_12,
       tab3.avg_total AS total_13
FROM tab1 
INNER JOIN tab2 ON tab1.country = tab2.country
INNER JOIN tab3 ON tab1.country = tab3.country
ORDER BY  tab1.avg_total DESC;
